In [1]:
# Experiment Preparation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

startdate_training = '2012-04-01'
enddate_training = '2020-05-01'

datasource = 'SPY'
data = yf.download(datasource,start = startdate_training,end = enddate_training)

[*********************100%***********************]  1 of 1 completed


In [2]:
def data_shift(data,target):
    data_shifted = data.drop([target],axis=1).shift()
    data_preprocessed = pd.concat([data_shifted,data[target]], axis=1)
    data_preprocessed.dropna(inplace=True)
    return data_preprocessed

In [3]:
data_preprocessed = data_shift(data,'Close')
data_preprocessed

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
2012-04-03,140.639999,142.210007,140.360001,116.761986,151741100.0,141.259995
2012-04-04,141.639999,141.880005,140.429993,116.284531,155806700.0,139.860001
2012-04-05,140.220001,140.339996,139.339996,115.132088,146896000.0,139.789993
2012-04-09,139.380005,140.199997,139.259995,115.074432,137439400.0,138.220001
2012-04-10,138.029999,139.839996,137.839996,113.782036,127555900.0,135.899994
...,...,...,...,...,...,...
2020-04-24,280.489990,283.940002,278.750000,269.941406,104709700.0,282.970001
2020-04-27,280.730011,283.700012,278.500000,273.704071,85063200.0,287.049988
2020-04-28,285.119995,288.269989,284.619995,277.650421,77896600.0,285.730011


In [4]:
best_lag_list = [39, 37, 27, 24, 23, 19, 14, 9, 5, 2]

In [5]:
def lag2shift(lag_list):
    shift_list = []
    max_lag = 40
    for i in lag_list:
        shift_list.append(max_lag - i)
    
    return shift_list

In [6]:
shift_list = lag2shift(best_lag_list)
shift_list

[1, 3, 13, 16, 17, 21, 26, 31, 35, 38]

In [7]:
def rearrange(X,lags):
    Re = np.empty((X.shape[0],0))
    target = X.iloc[:,-1]
    for i in lags:
        Re = np.concatenate((Re, X.shift(i)), axis = 1)                 
    Re = np.concatenate((Re, target.to_frame(name = 'Target')), axis = 1)
    Re = pd.DataFrame(Re).dropna()
    return Re

In [8]:
data_var = rearrange(data_preprocessed,shift_list)
data_var

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
38,132.630005,132.839996,131.419998,109.098045,167357600.0,132.100006,132.309998,133.229996,131.339996,108.826370,...,115.074432,137439400.0,138.220001,140.639999,142.210007,140.360001,116.761986,151741100.0,141.259995,133.699997
39,132.479996,132.850006,131.779999,108.744064,135465600.0,133.699997,131.250000,132.460007,129.990005,108.883995,...,113.782036,127555900.0,135.899994,141.639999,141.880005,140.429993,116.284531,155806700.0,139.860001,131.759995
40,133.160004,133.929993,131.169998,110.061165,152883500.0,131.759995,132.630005,132.839996,131.419998,109.098045,...,111.872200,235360300.0,137.000000,140.220001,140.339996,139.339996,115.132088,146896000.0,139.789993,131.470001
41,132.559998,133.690002,131.490005,108.464157,162370400.0,131.470001,132.479996,132.850006,131.779999,108.744064,...,112.777733,154133000.0,138.789993,139.380005,140.199997,139.259995,115.074432,137439400.0,138.220001,128.160004
42,131.710007,132.449997,130.339996,108.225464,196186000.0,128.160004,133.160004,133.929993,131.169998,110.061165,...,114.251221,154321500.0,137.139999,138.029999,139.839996,137.839996,113.782036,127555900.0,135.899994,128.100006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,278.350006,281.000000,276.910004,269.960785,92951600.0,279.079987,282.609985,286.790009,281.350006,272.369232,...,300.846252,176613400.0,302.459991,288.700012,297.890015,285.540009,284.883636,384975800.0,309.089996,282.970001
2028,280.489990,283.940002,278.750000,269.941406,104709700.0,282.970001,276.730011,278.040009,272.019989,264.099243,...,290.845612,186366800.0,297.459991,298.209991,309.160004,294.459991,297.220978,238703600.0,300.239990,287.049988
2029,280.730011,283.700012,278.500000,273.704071,85063200.0,287.049988,278.350006,281.000000,276.910004,269.960785,...,286.037598,228667200.0,274.230011,309.500000,313.839996,297.570007,288.710846,300139100.0,312.859985,285.730011
2030,285.119995,288.269989,284.619995,277.650421,77896600.0,285.730011,280.489990,283.940002,278.750000,269.941406,...,263.699646,309417300.0,288.420013,306.119995,313.100006,303.329987,300.846252,176613400.0,302.459991,293.209991


In [9]:
startdate_testing = '2020-05-01'
enddate_testing = '2022-06-01'
data_1 = yf.download(datasource,start = startdate_testing,end = enddate_testing)

[*********************100%***********************]  1 of 1 completed


In [10]:
data_preprocessed_1 = data_shift(data_1,'Close')

In [11]:
data_var_1 = rearrange(data_preprocessed_1,shift_list)
data_var_1

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
38,313.489990,314.500000,311.609985,303.160034,68471200.0,304.089996,314.170013,314.380005,306.529999,299.847198,...,276.818604,79569900.0,284.250000,291.709991,293.320007,288.589996,280.968140,122901700.0,282.790009,307.350006
39,309.839996,310.510010,302.100006,295.426819,132813500.0,307.350006,307.989990,311.049988,306.750000,301.770752,...,274.942200,73632600.0,287.679993,285.309998,290.660004,281.519989,273.529938,125180000.0,283.570007,300.049988
40,303.470001,307.640015,301.279999,298.593994,89468000.0,300.049988,313.489990,314.500000,311.609985,303.160034,...,278.259796,75250400.0,292.440002,280.739990,283.899994,279.130005,274.284393,80873200.0,286.190002,304.459991
41,306.160004,306.390015,299.420013,291.501892,127961000.0,304.459991,309.839996,310.510010,302.100006,295.426819,...,282.863922,76452400.0,292.500000,286.640015,289.250000,283.709991,276.818604,79569900.0,284.250000,308.359985
42,301.410004,304.609985,298.929993,295.786285,79773300.0,308.359985,303.470001,307.640015,301.279999,298.593994,...,282.921967,79514200.0,286.670013,288.040009,288.459991,283.779999,274.942200,73632600.0,287.679993,310.519989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,393.250000,397.029999,380.540009,387.954163,131432200.0,396.920013,403.500000,403.799988,390.549988,390.174530,...,450.971954,89048800.0,456.799988,460.019989,462.070007,457.179993,459.564789,86581500.0,458.700012,393.890015
521,392.829987,397.730011,390.380005,395.212799,76414900.0,393.890015,388.619995,394.140015,387.109985,387.784882,...,454.835236,59601000.0,451.029999,460.339996,461.200012,456.470001,456.727081,79666900.0,451.640015,397.369995
522,392.559998,395.149994,386.959991,392.195831,91448800.0,397.369995,393.250000,397.029999,380.540009,387.954163,...,449.090057,74214500.0,446.519989,457.890015,458.760010,451.160004,449.697449,121699900.0,452.920013,405.309998
523,392.309998,399.450012,391.890015,395.660858,91472900.0,405.309998,392.829987,397.730011,390.380005,395.212799,...,444.599457,106898000.0,448.769989,453.309998,453.459991,449.140015,450.971954,89048800.0,456.799988,415.260010


In [12]:
# 1994 + 487
# Covariance matrix

data_var_all = pd.concat([data_var,data_var_1], axis=0)

from pathlib import Path
filepath = Path('C:/Users/82422/OneDrive/internship/Optimal_Lag/Data/Data_VAR.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data_var_all.to_csv(filepath)